# 🎻 Proyecto Integrador Nivel 3: Cortex Orchestrator (v0.3)

En el Nivel 2, Cortex reaccionaba a comandos uno por uno. Pero un Agente IA real no trabaja así. Un agente real recibe una meta compleja (ej: "Escribe un informe de ventas") y debe descomponerla en múltiples subtareas que tienen **Dependencias** y **Prioridades**.

En este proyecto, construiremos el **Motor de Orquestación** de Cortex.

## El Problema: El Infierno de las Dependencias
Imagina que le pides al agente: "Analiza los datos y envíame un correo".
1.  Tarea A: Cargar Datos.
2.  Tarea B: Enviar Correo.

El agente **no puede** hacer B antes que A.
Si usamos una lista simple, podríamos equivocarnos en el orden. Necesitamos un **DAG (Directed Acyclic Graph)**.

## La Solución: Grafos + Colas de Prioridad
Nuestro sistema `WorkflowEngine` combinará dos estructuras de datos poderosas:
1.  **Grafo de Dependencias:** Para saber *qué* tarea desbloquea a cuál (A -> B).
2.  **Cola de Prioridad (Heap):** Para saber, entre todas las tareas posibles, cuál es la más urgente (ej: una alerta de seguridad va antes que un análisis rutinario).

## Herramientas Python Utilizadas

### 1. `@dataclass` (Data Classes)
Introducido en Python 3.7, es una forma moderna y limpia de definir clases que sirven principalmente para guardar datos.

* **Lo que nos ahorra:** Genera automáticamente el `__init__` (constructor), `__eq__` (para comparar igualdad) y un `__repr__` por defecto.
* **En nuestro código:** Aprovechamos que nos escribe el `__init__` automáticamente, pero decidimos **personalizar manualmente** el `__repr__` para que los mensajes en consola sean más cortos y legibles (mostrando solo ID y Prioridad en lugar de todo el objeto).

### 2. `heapq` (Algoritmo de Montículo)
Python no tiene una clase "PriorityQueue" por defecto que sea iterable fácilmente, pero tiene el módulo `heapq`.
Este módulo reorganiza una lista para que el elemento más pequeño (mayor prioridad numérica) siempre esté en la posición 0. Esto hace que encontrar la "siguiente tarea urgente" sea extremadamente rápido ($O(1)$).

### 3. Conjuntos (`set`)
Usaremos conjuntos para manejar las dependencias. La operación `subset` (subconjunto) es perfecta para preguntar: *"¿Están todas las dependencias de esta tarea dentro del grupo de tareas terminadas?"*.

## Lógica del Motor (Engine Logic)

El método `run()` implementa un bucle inteligente que sigue estos pasos hasta que no quedan tareas:

1.  **Check de Dependencias:** Revisa todas las tareas pendientes. Si las dependencias de una tarea ya están en el conjunto `completed_tasks`, esa tarea se mueve a la `ready_queue` (Cola de Listos).
2.  **Selección por Prioridad:** El `heap` nos da automáticamente la tarea más importante de la `ready_queue`.
3.  **Ejecución:** Simulamos el trabajo (en el futuro, aquí llamaremos a la API de OpenAI o Pandas).
4.  **Actualización de Estado:** Marcamos la tarea como completada, lo que podría desbloquear nuevas tareas en la siguiente vuelta del bucle.



Este flujo garantiza que **nunca** se ejecute una tarea antes de tiempo y que **siempre** se atienda lo urgente primero.

In [2]:
import heapq
import time
from dataclasses import dataclass, field
from typing import List, Set, Dict

# --- ESTRUCTURA DE DATOS: TAREA ---
@dataclass(order=True) # order=True permite comparar tareas por prioridad automáticamente
class Task:
    priority: int # 1 = Urgente, 10 = Baja
    id: str = field(compare=False)
    description: str = field(compare=False)
    dependencies: Set[str] = field(default_factory=set, compare=False) # IDs de tareas que deben terminar antes

    def __repr__(self):
        return f"[Task {self.id} | Prio: {self.priority}]"

# --- MOTOR DE ORQUESTACIÓN ---
class WorkflowEngine:
    def __init__(self):
        # Min-Heap para tareas listas para ejecutarse (ordenadas por prioridad)
        self.ready_queue = []
        # Diccionario para almacenar todas las tareas (Grafo implícito)
        self.all_tasks: Dict[str, Task] = {}
        # Conjunto de tareas completadas
        self.completed_tasks: Set[str] = set()

    def add_task(self, task: Task):
        """Registra una tarea en el sistema."""
        self.all_tasks[task.id] = task
        print(f"📝 Tarea Registrada: {task.id} (Dependencias: {task.dependencies})")

    def _check_dependencies(self):
        """Revisa qué tareas pendientes ya tienen sus dependencias resueltas."""
        for task_id, task in self.all_tasks.items():
            # Si la tarea no está completada Y no está ya en la cola de listos
            if task_id not in self.completed_tasks and task not in self.ready_queue:
                # Verificar si todas sus dependencias están en completed_tasks
                # issubset verifica si dependencies está contenido en completed_tasks
                if task.dependencies.issubset(self.completed_tasks):
                    heapq.heappush(self.ready_queue, task)
                    # Nota: En un sistema real, marcaríamos la tarea como 'queued' para no duplicarla

    def run(self):
        """Ciclo principal de ejecución."""
        print("\n🚀 INICIANDO CORTEX ORCHESTRATOR...")

        while len(self.completed_tasks) < len(self.all_tasks):
            # 1. Buscar tareas desbloqueadas
            self._check_dependencies()

            if not self.ready_queue:
                print("⚠️ Deadlock detectado o esperando eventos externos. No hay tareas listas.")
                break

            # 2. Obtener la tarea de mayor prioridad (Menor número)
            current_task = heapq.heappop(self.ready_queue)

            # Verificar doble check por seguridad (si usamos threads en el futuro)
            if current_task.id in self.completed_tasks:
                continue

            # 3. Ejecutar Tarea
            print(f"\n⚙️ Ejecutando: {current_task.description}...")
            time.sleep(2) # Simulación de trabajo

            # 4. Marcar como completada
            self.completed_tasks.add(current_task.id)
            print(f"✅ Completado: {current_task.id}")

        print("\n🏁 FLUJO DE TRABAJO FINALIZADO.")

# --- PRUEBA DEL SISTEMA ---
if __name__ == "__main__":
    engine = WorkflowEngine()

    # Definición de un DAG (Directed Acyclic Graph) de tareas
    # Flujo: [Ingesta] -> [Limpieza] -> [Análisis] -> [Reporte]
    #                      ^ [Security Check] (Prioridad Alta)

    t1 = Task(id="INGEST", priority=5, description="Cargar CSV de notas")

    t2 = Task(id="CLEAN", priority=5, description="Limpiar valores nulos", dependencies={"INGEST"})

    # Tarea urgente que depende de la ingesta pero debe pasar antes que el análisis
    t3 = Task(id="SEC_SCAN", priority=1, description="Escanear datos sensibles (GDPR)", dependencies={"INGEST"})

    t4 = Task(id="MODEL", priority=5, description="Entrenar modelo predictivo", dependencies={"CLEAN", "SEC_SCAN"})

    t5 = Task(id="REPORT", priority=5, description="Generar PDF final", dependencies={"MODEL"})

    # Agregamos desordenados para probar que el sistema ordena
    engine.add_task(t5)
    engine.add_task(t3)
    engine.add_task(t1)
    engine.add_task(t2)
    engine.add_task(t4)

    engine.run()

📝 Tarea Registrada: REPORT (Dependencias: {'MODEL'})
📝 Tarea Registrada: SEC_SCAN (Dependencias: {'INGEST'})
📝 Tarea Registrada: INGEST (Dependencias: set())
📝 Tarea Registrada: CLEAN (Dependencias: {'INGEST'})
📝 Tarea Registrada: MODEL (Dependencias: {'CLEAN', 'SEC_SCAN'})

🚀 INICIANDO CORTEX ORCHESTRATOR...

⚙️ Ejecutando: Cargar CSV de notas...
✅ Completado: INGEST

⚙️ Ejecutando: Escanear datos sensibles (GDPR)...
✅ Completado: SEC_SCAN

⚙️ Ejecutando: Limpiar valores nulos...
✅ Completado: CLEAN

⚙️ Ejecutando: Entrenar modelo predictivo...
✅ Completado: MODEL

⚙️ Ejecutando: Generar PDF final...
✅ Completado: REPORT

🏁 FLUJO DE TRABAJO FINALIZADO.
